# Cuaderno 02: Análisis Morfológico Integral de Astrocitos

**Objetivo:** Este cuaderno carga las máscaras de astrocitos finales y realiza un análisis cuantitativo completo para caracterizar su morfología y complejidad estructural.

**Flujo de Trabajo:**
1.  **Configuración**: Definir rutas, parámetros de calibración y análisis.
2.  **Carga de Datos**: Cargar la máscara de astrocitos y el canal GFAP original.
3.  **Análisis Morfológico por Astrocito**: Iterar sobre cada astrocito para calcular:
    * **Propiedades Básicas**: Volumen y centroide.
    * **Skeletonización 3D**: Obtener el esqueleto de los procesos.
    * **Análisis de Ramificaciones**: Medir longitud y número de ramas del esqueleto.
    * **Análisis de Sholl**: Medir la complejidad radial.
    * **Territorio Celular**: Calcular el volumen de la envolvente convexa.
4.  **Consolidación y Glosario**: Agrupar todos los resultados y explicar cada métrica.
5.  **Visualización**: Mostrar los resultados en Napari y con gráficos.
6.  **Guardado Final**: Exportar todos los datos a un único archivo CSV.

## Paso 0: Activar el Backend Gráfico

In [3]:
%gui qt
%pip install skan --quiet

Note: you may need to restart the kernel to use updated packages.


## Paso 1: Configuración de Rutas y Parámetros

In [4]:
from pathlib import Path

# --- Rutas de Archivos de Entrada ---
project_root = Path.cwd().parent
base_filename = "Inmuno 26-07-23.lif - CTL 1-2 a"
subfolder = "CTL/CTL 1-2"
original_image_path = project_root / f"data/raw/{subfolder}/{base_filename}.tif"
final_mask_path = project_root / f"data/processed/{base_filename}/04_final_astrocytes_mask.tif"

# --- Rutas de Archivos de Salida ---
results_dir = project_root / "results/tables"
results_dir.mkdir(parents=True, exist_ok=True)
morphology_output_path = results_dir / f"{base_filename}_full_morphology.csv"
skeleton_output_dir = project_root / "data/processed" / base_filename / "skeletons"
skeleton_output_dir.mkdir(parents=True, exist_ok=True)

# --- Parámetros de Calibración Física (µm)---
PIXEL_WIDTH_UM = 0.3788
PIXEL_HEIGHT_UM = 0.3788
Z_SPACING_UM = 1.0071
VOXEL_VOLUME_UM3 = PIXEL_WIDTH_UM * PIXEL_HEIGHT_UM * Z_SPACING_UM
PHYSICAL_SCALE = (Z_SPACING_UM, PIXEL_HEIGHT_UM, PIXEL_WIDTH_UM)

# --- Parámetros para el Análisis de Sholl ---
SHOLL_START_RADIUS_UM = 5.0
SHOLL_END_RADIUS_UM = 50.0
SHOLL_STEP_UM = 2.0

## Paso 2: Carga de Datos e Importaciones

In [6]:
import tifffile
import numpy as np
import pandas as pd
from skimage.measure import regionprops, label
# --- CORRECCIÓN AQUÍ ---
from skimage.morphology import skeletonize
# -----------------------
from skimage.filters import threshold_otsu
from scipy.spatial import ConvexHull
import skan
import napari
import matplotlib.pyplot as plt

try:
    astrocyte_mask = tifffile.imread(final_mask_path)
    original_image = tifffile.imread(original_image_path)
    gfap_channel = original_image[:, 1, :, :]
    astrocyte_ids = np.unique(astrocyte_mask)[1:]
    print(f"Datos cargados. Se analizarán {len(astrocyte_ids)} astrocitos.")
except FileNotFoundError:
    print("Error: No se encontraron archivos. Ejecuta el cuaderno 01 primero.")

Datos cargados. Se analizarán 56 astrocitos.


## Paso 3: Análisis Morfológico por Astrocito

Iteramos a través de cada astrocito para calcular un perfil morfológico completo, desde su volumen hasta la complejidad de sus ramas.

In [ ]:
all_morphology_results = []
all_skeletons = np.zeros_like(astrocyte_mask, dtype=np.uint8)
all_sholl_results = []

# Binarizamos el canal GFAP una sola vez para todos los análisis
gfap_threshold = threshold_otsu(gfap_channel)
binarized_gfap = gfap_channel > gfap_threshold

print("Iniciando análisis morfológico para cada astrocito...")
for i, astro_id in enumerate(astrocyte_ids):
    # --- 1. Aislar el astrocito y calcular propiedades básicas ---
    single_astrocyte_mask_bool = (astrocyte_mask == astro_id)
    single_astrocyte_mask_labeled = label(single_astrocyte_mask_bool)
    props = regionprops(single_astrocyte_mask_labeled)[0]
    volume_voxels = props.area
    centroid_z, centroid_y, centroid_x = props.centroid
    
    # --- 2. Skeletonización y Análisis de Ramas ---
    astrocyte_processes = binarized_gfap & single_astrocyte_mask_bool
    total_length_um = 0
    num_branches = 0
    
    # FIX 1: Comprobar si hay vóxeles para skeletonizar
    if np.any(astrocyte_processes):
        skeleton = skeletonize(astrocyte_processes).astype(np.uint8)
        
        # Comprobar si el esqueleto resultante no está vacío
        if np.any(skeleton):
            all_skeletons += skeleton * (i + 1)
            
            # FIX 2: Añadir el argumento 'separator' para silenciar la advertencia
            branch_data = skan.summarize(
                skan.Skeleton(skeleton, spacing=PHYSICAL_SCALE), 
                separator='_'
            )
            
            total_length_um = branch_data['branch_distance'].sum()
            num_branches = len(branch_data)

    # --- 3. Análisis de Sholl ---
    radii_um = np.arange(SHOLL_START_RADIUS_UM, SHOLL_END_RADIUS_UM, SHOLL_STEP_UM)
    z, y, x = np.indices(astrocyte_mask.shape)
    distances_um = np.sqrt(
        ((z - centroid_z) * Z_SPACING_UM)**2 +
        ((y - centroid_y) * PIXEL_HEIGHT_UM)**2 +
        ((x - centroid_x) * PIXEL_WIDTH_UM)**2
    )
    for j, r_um in enumerate(radii_um):
        r_inner_um = radii_um[j-1] if j > 0 else 0
        shell_mask = (distances_um > r_inner_um) & (distances_um <= r_um)
        intersection_mask = binarized_gfap & shell_mask
        _, num_intersections = label(intersection_mask, return_num=True)
        all_sholl_results.append({'label': astro_id, 'radius_um': r_um, 'intersections': num_intersections})

    # --- 4. Cálculo del Territorio (Convex Hull) ---
    points = np.argwhere(single_astrocyte_mask_bool)
    territory_volume_um3 = 0
    if len(points) > 3:
        points_um = points * np.array(PHYSICAL_SCALE)
        hull = ConvexHull(points_um)
        territory_volume_um3 = hull.volume
        
    # --- 5. Consolidar resultados para este astrocito ---
    all_morphology_results.append({
        'label': astro_id,
        'volume_um3': volume_voxels * VOXEL_VOLUME_UM3,
        'total_process_length_um': total_length_um,
        'num_branches': num_branches,
        'territory_volume_um3': territory_volume_um3,
        'sholl_max_intersections': pd.DataFrame(all_sholl_results).query(f'label == {astro_id}')['intersections'].max()
    })
    print(f"Analizado astrocito {astro_id} ({i+1}/{len(astrocyte_ids)})")

# --- 6. Crear DataFrames finales ---
morphology_df = pd.DataFrame(all_morphology_results)
sholl_df = pd.DataFrame(all_sholl_results)

# Guardamos la imagen con todos los esqueletos
tifffile.imwrite(skeleton_output_dir / f"{base_filename}_skeletons.tif", all_skeletons)

print("\nAnálisis morfológico completado.")
display(morphology_df.head())

Iniciando análisis morfológico para cada astrocito...


KeyError: 'branch-distance'

## Paso 4: Glosario de Resultados

A continuación se describe el significado de cada métrica calculada en la tabla `morphology_df`.

* `label`: El identificador numérico único para cada astrocito.
* `volume_um3`: **Volumen del Cuerpo Celular (µm³)**. Es el volumen del núcleo segmentado por Cellpose. No incluye los procesos.
* `total_process_length_um`: **Longitud Total de Procesos (µm)**. La suma de las longitudes de todas las ramas del esqueleto. Mide la extensión total del "cableado" del astrocito.
* `num_branches`: **Número de Ramas**. El número de segmentos que componen el esqueleto. Un mayor número sugiere una mayor complejidad.
* `territory_volume_um3`: **Volumen del Territorio (µm³)**. El volumen de la envolvente convexa que encierra a toda la célula. Representa el espacio de influencia total del astrocito.
* `sholl_max_intersections`: **Máximo de Intersecciones de Sholl**. El número pico de intersecciones encontradas en el análisis de Sholl, que indica la zona de mayor complejidad de ramificación.

## Paso 5: Visualización de Resultados

Visualizamos los resultados en Napari para una inspección cualitativa y creamos un gráfico de Sholl para un astrocito de ejemplo.

In [ ]:
# Cargamos los esqueletos que guardamos
skeletons_img = tifffile.imread(skeleton_output_dir / f"{base_filename}_skeletons.tif")

viewer = napari.Viewer()
# Añadimos el canal GFAP y la máscara de astrocitos
viewer.add_image(gfap_channel, name='GFAP', colormap='green', scale=PHYSICAL_SCALE)
viewer.add_labels(astrocyte_mask, name='Astrocitos', scale=PHYSICAL_SCALE, opacity=0.3)
# Añadimos los esqueletos 3D
viewer.add_labels(skeletons_img, name='Esqueletos 3D', scale=PHYSICAL_SCALE)

# --- Gráfico de Sholl para un astrocito de ejemplo ---
if not sholl_df.empty:
    astrocyte_to_plot = morphology_df['label'].iloc[0]
    sholl_subset = sholl_df[sholl_df['label'] == astrocyte_to_plot]
    
    plt.figure(figsize=(8, 6))
    plt.plot(sholl_subset['radius_um'], sholl_subset['intersections'], marker='o', linestyle='-')
    plt.title(f'Análisis de Sholl para Astrocito #{astrocyte_to_plot}')
    plt.xlabel('Radio desde el Centroide (µm)')
    plt.ylabel('Número de Intersecciones')
    plt.grid(True)
    plt.show()

Iniciando análisis de Sholl...
Análisis de Sholl completado.


,label,radius_um,intersections
0,44,5.0,4
1,44,7.0,13
2,44,9.0,12
3,44,11.0,11
4,44,13.0,20


## Paso 5: Guardado de Resultados Cuantitativos

Ahora guardamos ambos DataFrames (la cuantificación básica y el análisis de Sholl detallado) en archivos CSV.

In [18]:
if 'quant_df' in locals() and not quant_df.empty:
    quant_df.to_csv(quantification_output_path, index=False)
    print(f"Resultados de cuantificación guardados en: {quantification_output_path}")

if 'sholl_df' in locals() and not sholl_df.empty:
    sholl_df.to_csv(sholl_output_path, index=False)
    print(f"Resultados del análisis de Sholl guardados en: {sholl_output_path}")

Resultados de cuantificación guardados en: /home/daniel/Proyectos/astrocitos-3d-analysis/results/tables/Inmuno 26-07-23.lif - CTL 1-2 a_quantification.csv
Resultados del análisis de Sholl guardados en: /home/daniel/Proyectos/astrocitos-3d-analysis/results/tables/Inmuno 26-07-23.lif - CTL 1-2 a_sholl_analysis.csv


## Paso 6: Guardado Final de Datos

Guardamos la tabla morfológica completa y los datos detallados de Sholl en archivos CSV separados.

In [ ]:
if 'morphology_df' in locals() and not morphology_df.empty:
    morphology_df.to_csv(morphology_output_path, index=False)
    print(f"Resultados morfológicos guardados en: {morphology_output_path}")

if 'sholl_df' in locals() and not sholl_df.empty:
    sholl_df.to_csv(sholl_output_path, index=False)
    print(f"Resultados del análisis de Sholl guardados en: {sholl_output_path}")